# RAG Agent


## 1. Setup


In [38]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import nest_asyncio
nest_asyncio.apply()

import warnings
import logging
from IPython.display import clear_output

warnings.filterwarnings('ignore')
logging.getLogger().setLevel(logging.ERROR)

In [2]:
from llama_index.core import (
    PromptTemplate,
    Settings,
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.response_synthesizers import ResponseMode

from llama_index.core.tools import QueryEngineTool, ToolMetadata
import sys
from loguru import logger

logger.add(sys.stdout, level="INFO", format="{time} {level} {message}", colorize=True)
logger.add("logs/nb_log_{time}.log", 
           rotation="10 MB",  # Rotate log file when it reaches 10MB
           retention="30 days",  # Keep logs for 30 days
           compression="zip",  # Optionally compress old logs to save space
           format="{time} {level} {message} {extra}")

2

In [4]:
llm = Ollama(model="llama3.2:3b-instruct-q8_0", temperature=0.01)
embedding = OllamaEmbedding(model_name="mxbai-embed-large")

Settings.llm = llm
Settings.embed_model = embedding

hello_embedding = embedding.get_text_embedding("hello")
print(len(hello_embedding))

1024


## 2. Load Data


In [5]:
index_loaded = False
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="../storage/q1"
    )
    q1_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="../storage/q2"
    )
    q2_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="../storage/q3"
    )
    q3_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    print("Failed to load indices")

Failed to load indices


In [6]:
if not index_loaded:
    # load data
    q1_docs = SimpleDirectoryReader(
        input_files=["../data/crm-2025-10Q-q1.pdf"]
    ).load_data()
    q2_docs = SimpleDirectoryReader(
        input_files=["../data/crm-2025-10Q-q2.pdf"]
    ).load_data()
    q3_docs = SimpleDirectoryReader(
        input_files=["../data/crm-2025-10Q-q3.pdf"]
    ).load_data()

    # build index
    q1_index = VectorStoreIndex.from_documents(
        q1_docs,
    )
    q2_index = VectorStoreIndex.from_documents(
        q2_docs,
    )
    q3_index = VectorStoreIndex.from_documents(
        q3_docs,
    )

In [7]:
# persist index
q1_index.storage_context.persist(persist_dir="../storage/q1")
q2_index.storage_context.persist(persist_dir="../storage/q2")
q3_index.storage_context.persist(persist_dir="../storage/q3")

## 3. Setup Indices & Query Engines


In [8]:
q1_engine = q1_index.as_query_engine(
    similarity_top_k=5,
    response_mode=ResponseMode.TREE_SUMMARIZE,
)
q2_engine = q2_index.as_query_engine(
    similarity_top_k=5,
    response_mode=ResponseMode.TREE_SUMMARIZE,
)
q3_engine = q3_index.as_query_engine(
    similarity_top_k=5,
    response_mode=ResponseMode.TREE_SUMMARIZE,
)
q1_response = q1_engine.query("What is Salesforce's revenue for the first quarter of FY25?")
print(q1_response)

$9,133 million.


## 4. Tools Setup


In [9]:
from llama_index.core.tools import QueryEngineTool

query_tool_q1 = QueryEngineTool.from_defaults(
    query_engine=q1_engine,
    name="q1_fy25",
    description="Use this tool to query the first quarter of FY25 financial data for Salesforce",
)

In [10]:

from typing import Optional

from llama_index.core.tools import FunctionTool
from llama_index.core.bridge.pydantic import BaseModel

# we will store booking under random IDs
bookings = {}

In [12]:
class Booking(BaseModel):
    name: Optional[str] = None
    email: Optional[str] = None
    phone: Optional[str] = None
    date: Optional[str] = None
    time: Optional[str] = None

def get_booking_state(user_id: str) -> str:
    if user_id not in bookings:
        return f"No booking found for user {user_id}"
    return str(bookings[user_id])

def update_booking(user_id: str, property: str, value: str) -> str:
    if user_id not in bookings:
        return f"No booking found for user {user_id}"
    bookings[user_id][property] = value
    return f"Booking updated for user {user_id} with {property} = {value}"

def create_booking(user_id: str) -> str:
    bookings[user_id] = Booking()
    return f"Booking created, but not yet confirmed. Please provide your name, email, phone, date, and time."

def cancel_booking(user_id: str) -> str:
    if user_id not in bookings:
        return f"No booking found for user {user_id}"
    del bookings[user_id]
    
def confirm_booking(user_id: str) -> str:
    booking = bookings[user_id]
    if booking.name is None:
        raise ValueError("Please provide your name   first.")
    if booking.email is None:
        raise ValueError("Please provide a valid email first.")
    if booking.phone is None:
        raise ValueError("Please provide a valid phone number first.")
    if booking.date is None:
        raise ValueError("Please provide a date first.")
    if booking.time is None:
        raise ValueError("Please provide a time between 9am and 5pm first.")
    return (
        f"Booking confirmed for user {user_id} with name {booking.name}, "
        f"email {booking.email}, phone {booking.phone}, "
        f"date {booking.date}, and time {booking.time}"
    )

In [13]:
get_booking_state_tool = FunctionTool.from_defaults(
    fn=get_booking_state,
    name="get_booking_state",
    description="Use this tool to get the current state of a booking.",
)

update_booking_tool = FunctionTool.from_defaults(
    fn=update_booking,
    name="update_booking",
    description="Use this tool to update a booking.",
)

create_booking_tool = FunctionTool.from_defaults(
    fn=create_booking,
    name="create_booking",
    description="Use this tool to create a booking.",
    return_direct=True,
)

confirm_booking_tool = FunctionTool.from_defaults(
    fn=confirm_booking,
    name="confirm_booking",
    description="Use this tool to confirm a booking.",
    return_direct=True,
)

cancel_booking_tool = FunctionTool.from_defaults(
    fn=cancel_booking,
    name="cancel_booking",
    description="Use this tool to cancel a booking.",
    return_direct=True,
)

## 5. Testing with a Customer


In [23]:

from llama_index.llms.ollama import Ollama
from llama_index.core.llms import ChatMessage
from llama_index.core.agent import FunctionCallingAgent

llm = Ollama(model="llama3.2:3b-instruct-q8_0", temperature=0.1)

prefix_messages = [
    ChatMessage(
        role="system",
        content=(
            f"You are now connected to the booking system and helping a customer with a booking. "
            "Only enter details that the user has explicitly provided. "
            "Do not make up any details."
        ),
    )
]

agent = FunctionCallingAgent.from_tools(
    tools=[
        get_booking_state_tool,
        update_booking_tool,
        create_booking_tool,
        confirm_booking_tool,
        cancel_booking_tool,
    ],
    llm=llm,
    prefix_messages=prefix_messages,
    max_function_calls=10,
    allow_parallel_tool_calls=False,
    verbose=True,
)

In [24]:
response = agent.chat("Hello! I would like to make a booking, around 5pm?")

> Running step d893d917-9d3e-4cc1-b10b-d1c15ea04aac. Step input: Hello! I would like to make a booking, around 5pm?
Added user message to memory: Hello! I would like to make a booking, around 5pm?
=== Calling Function ===
Calling function: create_booking with args: {"user_id": "around 5pm"}
=== Function Output ===
Booking created, but not yet confirmed. Please provide your name, email, phone, date, and time.


In [25]:
response = agent.chat("Sure! My name is John, and my email is johndoe@gmail.com")

> Running step dc6b232d-6215-4eb4-836f-8081971d8dbf. Step input: Sure! My name is John, and my email is johndoe@gmail.com
Added user message to memory: Sure! My name is John, and my email is johndoe@gmail.com
=== Calling Function ===
Calling function: create_booking with args: {"user_id": "{\"name\": \"John\", \"email\": \"johndoe@gmail.com\"}"}
=== Function Output ===
Booking created, but not yet confirmed. Please provide your name, email, phone, date, and time.


In [26]:
response = agent.chat(
    "Right! My phone number is 1234567890, the date of the booking is April 5, at 5pm."
)

> Running step 757d0e1b-9fee-4b6b-99d7-4c79369f5f4c. Step input: Right! My phone number is 1234567890, the date of the booking is April 5, at 5pm.
Added user message to memory: Right! My phone number is 1234567890, the date of the booking is April 5, at 5pm.
=== Calling Function ===
Calling function: create_booking with args: {"user_id": "{\"name\": \"John\", \"email\": \"johndoe@gmail.com\", \"phone\": \"1234567890\", \"date\": \"April 5\", \"time\": \"5pm\"}"}
=== Function Output ===
Booking created, but not yet confirmed. Please provide your name, email, phone, date, and time.


In [27]:
response = agent.chat(
    "Right! My name is John Doe, my email is johndoe@gmail.com, "
    "my phone is 1234567890, the date of the booking is April 5, 2025 for time at 3pm."
)

> Running step 542449e4-ff64-448d-98c2-bdd9fcac6a76. Step input: Right! My name is John Doe, my email is johndoe@gmail.com, my phone is 1234567890, the date of the booking is April 5, 2025 for time at 3pm.
Added user message to memory: Right! My name is John Doe, my email is johndoe@gmail.com, my phone is 1234567890, the date of the booking is April 5, 2025 for time at 3pm.
=== Calling Function ===
Calling function: create_booking with args: {"user_id": "{\"name\": \"John Doe\", \"email\": \"johndoe@gmail.com\", \"phone\": \"1234567890\", \"date\": \"April 5, 2025\", \"time\": \"3pm\"}"}
=== Function Output ===
Booking created, but not yet confirmed. Please provide your name, email, phone, date, and time.


In [39]:
from llama_index.llms.openai import OpenAI
gpt4 = OpenAI(model="gpt-4", temperature=0.01)
gpt4_agent = FunctionCallingAgent.from_tools(
    tools=[
        get_booking_state_tool,
        update_booking_tool,
        create_booking_tool,
        confirm_booking_tool,
        cancel_booking_tool,
    ],
    llm=gpt4,
    prefix_messages=prefix_messages,
    max_function_calls=10,
    allow_parallel_tool_calls=False,
    verbose=True,
)

In [40]:
response = gpt4_agent.chat("Hello! I would like to make a booking, around 5pm?")

> Running step 72281cf8-850c-4fb7-87ce-8868a904828e. Step input: Hello! I would like to make a booking, around 5pm?
Added user message to memory: Hello! I would like to make a booking, around 5pm?
=== Calling Function ===
Calling function: create_booking with args: {"user_id": "1"}
=== Function Output ===
Booking created, but not yet confirmed. Please provide your name, email, phone, date, and time.


In [ ]:
response = gpt4_agent.chat("Sure! My userid is: 1234")

> Running step 8b62040a-567d-425e-965e-790f5d044d30. Step input: Sure! My userid is: 1234
Added user message to memory: Sure! My userid is: 1234
=== Calling Function ===
Calling function: create_booking with args: {"user_id": "1234"}
=== Function Output ===
Booking created, but not yet confirmed. Please provide your name, email, phone, date, and time.


In [37]:
response = gpt4o_agent.chat(
    "Right! My name is John, email is johndoe@gmail.com, " + \
    "phone is 1234567890, date is April 5, 2025, and time is 3pm."
)


> Running step 5a3ab26d-6196-435f-95bc-177e0e15ccfe. Step input: Right! My name is John, email is johndoe@gmail.com, phone is 1234567890, date is April 5, 2025, and time is 3pm.
Added user message to memory: Right! My name is John, email is johndoe@gmail.com, phone is 1234567890, date is April 5, 2025, and time is 3pm.
=== Calling Function ===
Calling function: update_booking with args: {"user_id": "1234", "property": "name", "value": "John"}
=== Function Output ===
Encountered error: 'Booking' object does not support item assignment
> Running step 78a44540-6434-48a2-a49a-c6cfa9a96434. Step input: None
=== Calling Function ===
Calling function: update_booking with args: {"user_id": "1234", "property": "email", "value": "johndoe@gmail.com"}
=== Function Output ===
Encountered error: 'Booking' object does not support item assignment
> Running step 8b2c0e4f-7457-45cd-907d-0c8d08103e9a. Step input: None


KeyboardInterrupt: 